In [10]:
%cd Medecin-AI-Chatbot/

[Errno 2] No such file or directory: 'Medecin-AI-Chatbot/'
/Users/i.g/Documents/Medecin-AI-Chatbot/research


/Users/i.g/Documents/Medecin-AI-Chatbot/AIChatBotVEnv/lib/python3.13/site-packages/IPython/core/magics/osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})


In [7]:
import os
from dotenv import load_dotenv
from langchain_openai import OpenAI
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
from langchain_pinecone import PineconeVectorStore

load_dotenv()

PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

MeinLLM = OpenAI(temperature=0.9, max_tokens=500)

In [8]:
def loadPdfDoc(data):
    loader = DirectoryLoader(data, glob='*.pdf', loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

In [9]:
extractedData = loadPdfDoc(data='Data/')

FileNotFoundError: Directory not found: 'Data/'

In [ ]:
# Datein in klieneren Chunks splitten:

def textSplit(extractedData):
    textSpitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap=20)
    textChunks = textSpitter.split_documents(extractedData)
    return textChunks

In [ ]:
textChunks = textSplit(extractedData=extractedData)
print("Length of text Chunks", len(textChunks))

In [ ]:
# Herunterladen von einem Embedding Model von HuggingFace:
def downloadHuggingFace_Embeddings():
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [ ]:
embeddings = downloadHuggingFace_Embeddings()

In [ ]:
# Eine Pinecone cluster erstellen:

pc =  Pinecone(api_key=PINECONE_API_KEY)

pc.create_index(
    name="medicin-chat-bot", # Ich hatte ein Problem hier: nach lange Suche --> Losüng: keine Uppercases sind in Pinecone Indexes erlaubt
    dimension=384, # diese Dimension entspricht die Dimension vom HugginfFace Model, es ist sehr wichtig auf diese Dimension zu achten (Falls zukünftig anderes Model)
    metric="cosine", # andere metrics z.B: euclidean ... 
    spec=ServerlessSpec(
        # diese Specs sind die einzigen die gratis sind
        cloud="aws",
        region="us-east-1"
    )
)

In [ ]:
# Die Chunks embedden --> Chunks sind jetzt Vektoren und eine VektorStore (DatenBank) ist dafür nötig --> PineconeVectorStore

docSearch = PineconeVectorStore.from_documents(
    documents=textChunks,
    index_name="medicin-chat-bot",
    embedding=embeddings
)

In [ ]:
# load existierendes Pinecone Index:

docSearch = PineconeVectorStore.from_existing_index(
    index_name="medicin-chat-bot",
    embedding=embeddings
)

In [ ]:
retriever = docSearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

# Hier wird der OpenAI LLM model benutzt, um die vernünftige, richige und gezielte Ergebnisse zu bekommen:

In [ ]:
System_Prompt = (
    "You are a medical assistant for question-answering tasks related to the medical field."
    "Use only the following pieces of retrieved context to answer the questions you get asked "
    "and nothing else. If you don't know the answer, say that you don't know."
    "Keep the answers concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", System_Prompt),
        ("human", "{input}")
    ]
)

In [ ]:
question_answer_chain = create_stuff_documents_chain(MeinLLM, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [ ]:
Answer = rag_chain.invoke({"input":"my fever is high, i am coughing the hole time, my nose is closed and i keep spitting mucus, what do i have and what medecine should i take"}) 
print(Answer["answer"])